<a href="https://colab.research.google.com/github/al34n1x/DataScience/blob/master/6.Gestion_de_datos/Agregaci%C3%B3n_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Agregación de datos y operaciones de grupo](#scrollTo=BFynk27lvXm6)

>>[Actividades que veremos en este apartado](#scrollTo=EMGwZJuAv306)

>>>[Mecánica del GroupBy](#scrollTo=F1ju7vADwWy1)

>>>[Seleccionando una columna o subset de columnas](#scrollTo=_SAtsTx-1xjS)

>>>[Agrupando con dicts y series](#scrollTo=SFpRZZb23nOj)

>>>[Agrupación con funciones](#scrollTo=hNvhRjQg5IhT)

>>>[Data Aggregation](#scrollTo=tCCi7PrE5ecH)

>>>[Aplicación de columna inteligente y de funciones múltiples](#scrollTo=rbeK_M1blP_0)

>>[Aplicar: general dividir-aplicar-combinar](#scrollTo=K8sgtonjqmcI)

>>>[Análisis de cuantiles y buckets](#scrollTo=mFfqx7P0sbeq)

>>>[Rellenar valores perdidos con valores específicos de grupo](#scrollTo=tvKXddD9tV0C)

>>[Muestreo aleatorio y permutación](#scrollTo=vTsm2BuHv_SC)

>>>[Promedio ponderado grupal y correlación](#scrollTo=eX6B4Plhxr8P)

>>[Pivot Tables y tabulación cruzada](#scrollTo=3e5jR8qh0UMW)

>>>[Tabulaciones cruzadas (crosstab)](#scrollTo=gprrjJ0m12nf)



# Agregación de datos y operaciones de grupo

La categorización de un conjunto de datos y la aplicación de una función a cada grupo, ya sea una agregación o transformación, es un componente crítico del trabajo de análisis de datos. Después de cargar, fusionar y preparar un conjunto de datos, es posible que debas calcular estadísticas de grupo o posiblemente tablas dinámicas para fines de informes o visualización. Pandas proporciona una interfaz de grupo flexible, que te permite cortar, y resumir conjuntos de datos de forma natural.

Como verás, con la expresividad de Python y pandas, podemos realizar operaciones grupales bastante complejas utilizando cualquier función que acepte un objeto Pandas o una matriz NumPy. 

## Actividades que veremos en este apartado

* Dividir un Dataframe en pedazos usando una o más claves (en forma de funciones, matrices o nombres de columna de DataFrame).

* Calcular estadísticas de resumen de grupo, como conteo, media o desviación estándar, o una función definida por el usuario.

* Aplicar transformaciones como normalización, regresión lineal, clasificación o selección de subconjuntos.

* Calcular tablas dinámicas y tabulaciones cruzadas.

* Realizar análisis de cuantiles y otros análisis de grupos estadísticos.

### Mecánica del GroupBy

Existe un término conocido entre los analistas que describe operaciones de grupo, *split-apply-combine*.

En la primera parte de este proceso dividimos dataframes o series (split) en grupos basados en una o más keys. Una vez realizado la división, realizamos la función *apply* a cada grupo, produciendo un nuevo valor.

Finalmente, tomamos el resultado de esas operaciones y las combinamos en un objeto.

![alt text](https://raw.githubusercontent.com/al34n1x/DataScience/master/img/split-apply-combine.png)

*Fuente: Python for Data Analysis, 2nd Edition*



In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.152568,-0.214047
1,a,two,-0.071531,-0.512542
2,b,one,0.627634,0.212258
3,b,two,1.787160,0.014875
4,a,one,-0.433680,-0.881778


Supongamos que deseas calcular la *media* de la columna data1 usando las etiquetas de key1

In [5]:
grouped = df['data1'].groupby(df['key1'])
print(grouped)

In [7]:
grouped.mean()

key1
a    0.215786
b    1.207397
Name: data1, dtype: float64

Aquí agrupamos los datos usando dos claves, y la Serie resultante ahora tiene un índice jerárquico.

In [8]:
media = df['data1'].groupby([df['key1'], df['key2']]).mean()
media

key1  key2
a     one     0.359444
      two    -0.071531
b     one     0.627634
      two     1.787160
Name: data1, dtype: float64

In [9]:
media.unstack()

key2,one,two
key1,,
a,0.359444,-0.071531
b,0.627634,1.787160


En el siguiente ejemplo todo el grupo de keys son series

In [10]:
prov = np.array(['Buenos Aires', 'Buenos Aires', 'Córdoba', 'Córdoba', 'Tucumán'])
anios = np.array([2005, 2005, 2005, 2006, 2006])

In [11]:
df['data1']

0    1.152568
1   -0.071531
2    0.627634
3    1.787160
4   -0.433680
Name: data1, dtype: float64

In [12]:
df['data1'].groupby([prov, anios]).mean()

Buenos Aires  2005    0.540519
Córdoba       2005    0.627634
              2006    1.787160
Tucumán       2006   -0.433680
Name: data1, dtype: float64

### Seleccionando una columna o subset de columnas

La indexación de un objeto **GroupBy** creado a partir de un DataFrame con un nombre de columna o matriz de nombres de columna, genera un subconjunto de columnas para la agregación.

In [14]:
df.groupby('key1')['data1'] # Equivalente a df['data1'].groupby(df['key1'])
df.groupby('key1')['data2'] # Equivalente a df[['data2']].groupby(df['key1'])

Especialmente para grandes conjuntos de datos, puede ser conveniente agregar solo unas pocas columnas. Por ejemplo, en el conjunto de datos anterior, para calcular promedios solo para la columna data2 y obtener el resultado como un DataFrame, podríamos escribir:

In [15]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.547912
     two  -0.512542
b    one   0.212258
     two   0.014875

El objeto devuelto por esta operación de indexación es un DataFrame agrupado.

Será una lista o matriz o una Serie agrupada si solo se pasa un solo nombre de columna como escalar

In [16]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped.mean()

key1  key2
a     one    -0.547912
      two    -0.512542
b     one     0.212258
      two     0.014875
Name: data2, dtype: float64

### Agrupando con dicts y series

Puede que necesites agrupar información existente en algo diferente a un arreglo. Consideremos el siguiente Dataframe:


In [17]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,1.250064,-0.378525,-0.662189,-0.939930,-1.202558
Steve,1.585349,-0.323111,0.889480,-1.754796,0.411560
Wes,1.467208,0.358257,1.167450,0.577029,0.315957
Jim,-1.298096,0.825799,-0.049621,-0.853500,1.779798
Travis,1.136364,0.392435,1.549098,2.055965,0.484460


In [18]:
people.iloc[2:3, [1, 2]] = np.nan # Agrega un par de NaN
people

,a,b,c,d,e
Joe,1.250064,-0.378525,-0.662189,-0.939930,-1.202558
Steve,1.585349,-0.323111,0.889480,-1.754796,0.411560
Wes,1.467208,NaN,NaN,0.577029,0.315957
Jim,-1.298096,0.825799,-0.049621,-0.853500,1.779798
Travis,1.136364,0.392435,1.549098,2.055965,0.484460


Supongamos que tenemos una lista de columnas que corresponden a ese Dataframe y queremos realizar una operacion **sum** entre las columnas por grupo

In [21]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

Ahora podemos construir un arreglo a partir del dict y se lo pasamos a la operación **groupby**, pero en cambio le pasamos directamente el dict como key.


In [22]:
by_column=people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-1.602119,-0.331019
Steve,-0.865316,1.673797
Wes,0.577029,1.783166
Jim,-0.903121,1.307501
Travis,3.605063,2.013259


### Agrupación con funciones
El uso de las funciones de Python es una forma más genérica de definir un mapeo de grupo en comparación con un dict o Series. 

**Cualquier función que se pase como clave de grupo se llamará una vez por valor de índice**, y los valores de retorno se utilizarán como nombres de grupo. Más concretamente, consideremos el DataFrame de ejemplo de la sección anterior, que tiene los nombres de las personas como valores de índice. Supongamos que deseas agrupar por la longitud de los nombres; Si bien podrías calcular una matriz de longitudes de cadena, es más simple simplemente pasar la función len:

In [25]:
people #Recordemos el Dataframe original

,a,b,c,d,e
Joe,1.250064,-0.378525,-0.662189,-0.939930,-1.202558
Steve,1.585349,-0.323111,0.889480,-1.754796,0.411560
Wes,1.467208,NaN,NaN,0.577029,0.315957
Jim,-1.298096,0.825799,-0.049621,-0.853500,1.779798
Travis,1.136364,0.392435,1.549098,2.055965,0.484460


In [28]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.419176,0.447274,-0.711810,-1.216401,0.893198
5,1.585349,-0.323111,0.889480,-1.754796,0.411560
6,1.136364,0.392435,1.549098,2.055965,0.484460




---


### Data Aggregation
Las agregaciones se refieren a cualquier transformación de datos que produce valores escalares a partir de matrices. Los ejemplos anteriores han utilizado varios de ellos, como el cálculo de promedio, la suma, etc. 


Function name |	Description
------------- | -----------
count	| Número de valores no-NA en el grupo
sum	| Suma de valores no-NA
mean	| Media de valores no-NA 
median	| Mediana aritmética de valores no-NA
std, var	| Desviación y varianza estándar imparcial (denominador n - 1)
min, max	| Mínimo y máximo de valores no-NA
prod	| Producto de valores no-NA 
first, last	| Primer y último valores no-NA 




Puedes usar agregaciones de tu propio diseño y, además, llamar a cualquier método que también esté definido en el objeto agrupado. Por ejemplo, puedes recordar que el cuantil calcula los cuantiles de muestra de una serie o las columnas de un marco de datos.

**Ejemplo**: Veamos por ejemplo un ejemplo de cálculo de cuantil sobre un dataframe agrupado.

Si bien el cuantil no se implementa explícitamente para GroupBy, es un método de la Serie y, por lo tanto, está disponible para su uso.  Internamente, GroupBy corta eficientemente la serie, llama a **quantile()** para cada pieza y luego ensambla esos resultados en el objeto de resultado:

In [29]:
df

,key1,key2,data1,data2
0,a,one,1.152568,-0.214047
1,a,two,-0.071531,-0.512542
2,b,one,0.627634,0.212258
3,b,two,1.787160,0.014875
4,a,one,-0.433680,-0.881778


In [31]:
grouped = df.groupby('key1')

In [34]:
grouped['data1'].quantile(0.5)
'''
Análisis del resultado:
1. Vemos que el cuantil del 50% para la clave "a", que tenia 3 valores, coincide con el valor del medio.
2. En el caso de la clave "b" que tenia dos valores, coincide con el promedio de ambos.
'''

key1
a   -0.071531
b    1.207397
Name: data1, dtype: float64

Puedes notar que algunos métodos como **describe** también funcionan, aunque no son agregaciones, estrictamente hablando

In [33]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.215786  0.831239 -0.433680 -0.252606 -0.071531  0.540519   
b      2.0  1.207397  0.819909  0.627634  0.917515  1.207397  1.497278   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.152568   3.0 -0.536122  0.334489 -0.881778 -0.697160 -0.512542   
b     1.787160   2.0  0.113566  0.139570  0.014875  0.064221  0.113566   

                          
           75%       max  
key1                      
a    -0.363294 -0.214047  
b     0.162912  0.212258

### Agregación de columna inteligente y de funciones múltiples

Volvamos al conjunto de datos de propinas de ejemplos anteriores. Después de cargarlo con read_csv, agregamos una columna de porcentaje de propina tip_pct

In [ ]:
prop = pd.read_csv('https://raw.githubusercontent.com/al34n1x/DataScience/master/6.Gestion_de_datos/tips.csv')
prop['tip_pct'] = prop['tip'] / prop ['total_bill']

In [36]:
prop[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


Como hemos visto, agregar una Serie o todas las columnas de un dataframe de datos es una cuestión de utilizar el agregado con la función deseada o llamar a un método como **mean** o **std**. 
Sin embargo, es posible que desees agregar usando una función diferente dependiendo de la columna, o múltiples funciones a la vez. 

In [37]:
grouped = prop.groupby(['day', 'smoker'])

In [38]:
grouped_pct = grouped['tip_pct']

Ten en cuenta que para estadísticas descriptivas como las de la Tabla que hemos compartido al comienzo, igual que cuando hicimos el agrupamiento por función **len** se puede pasar el nombre de la función como una cadena, en este caso **mean**

In [44]:
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

Una manera equivalente de escribir lo mismo es realizar una **agregación**, que agregará tantas columnas como se le indique a la funcion **.agg()**:

In [45]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [46]:
grouped_pct.agg(['min','max'])

min       max
day  smoker                    
Fri  No      0.120385  0.187735
     Yes     0.103555  0.263480
Sat  No      0.056797  0.291990
     Yes     0.035638  0.325733
Sun  No      0.059447  0.252672
     Yes     0.065660  0.710345
Thur No      0.072961  0.266312
     Yes     0.090014  0.241255

In [47]:
def peak_to_peak(arr):      # Función de agregación propia 
  return arr.max() - arr.min()

Si pasas una lista de funciones o nombres de funciones, obtiene un DataFrame con nombres de columnas tomados de las funciones.

In [50]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

¿Y si quisieramos ver también sobre cuántos datos se hace cada operación para cada key, qué agregaríamos?

In [ ]:
grouped_pct.agg(['mean', 'std', peak_to_peak]) # ¿Qué falta?



---
**Cambiar los nombres de columna resultantes de la agregación**:

No se necesita aceptar los nombres que GroupBy le da a las columnas. Si pasas una lista de tuplas (nombre, función), el primer elemento de cada tupla se usará como los nombres de columna de DataFrame.

In [52]:
grouped_pct.agg([('Promedio de tip%', 'mean'), ('Desvio de tip%', np.std)])

Promedio de tip%  Desvio de tip%
day  smoker                                  
Fri  No              0.151650        0.028123
     Yes             0.174783        0.051293
Sat  No              0.158048        0.039767
     Yes             0.147906        0.061375
Sun  No              0.160113        0.042347
     Yes             0.187250        0.154134
Thur No              0.160298        0.038774
     Yes             0.163863        0.039389

Con un DataFrame tienes más opciones, ya que puedes especificar una lista de funciones para aplicar a todas las columnas o diferentes funciones por columna.

Para comenzar, supongamos que deseamos calcular las mismas tres estadísticas para las columnas tip_pct y total_bill

In [60]:
columnas = ['tip_pct', 'total_bill'] # Ahora tenemos una lista de columnas a diferencia del ejemplo anterior donde solo seleccionabamos
# una columna y a esa columna le aplicabamos varias funciones
functions = ['count', 'mean', 'max'] # A cada una de las columnas de la lista le aplicaremos entonces varias funciones

result = grouped[columnas].agg(functions) # A las dos columnas del DF le aplicamos las tres funciones

result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

Ahora, supongamos que deseamos aplicar funciones potencialmente diferentes a una o más de las columnas. Para hacer esto, pasamos un dict a *agg* que contenga una asignación de nombres de columna a cualquiera de las especificaciones de funciones enumeradas hasta ahora

In [61]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [62]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40


---

## Apply

El método mas general de uso de GroupBy es **apply**.

Como se ilustra en la Figura, **apply** divide el objeto que se está manipulando en piezas, invoca la función pasada en cada pieza y luego intenta concatenar las piezas juntas.

![alt text](https://raw.githubusercontent.com/al34n1x/DataScience/master/img/split-apply-combine.png)

*Fuente: Python for Data Analysis, 2nd Edition*

Supongamos que deseamos seleccionar los cinco valores principales de **tip_pct** por grupo. Primero, escribimos una función que seleccione las filas con los valores más grandes en una columna particular:

In [65]:
# Volvamos a trabajar con el dataframe original
prop.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [72]:
def top(df, n=5, column='tip_pct'):
  return df.sort_values(by=column)[-n:] # Está haciendo un sort por columna "tip_pct" y retornando las últimas "n" filas

In [73]:
top(prop, n=6) # Llamada a la función top y reemplaza n=5 de la funcion por n=6

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


Ahora, si agrupamos por fumador, por ejemplo, y llamamos a esta función, obtenemos lo siguiente:

In [74]:
prop.groupby('smoker').apply(top) # apply llama a la función top

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

¿Qué ha pasado aquí? 

La función superior se llama en cada grupo de filas desde cada split del dataframe (el primer grupo es la agrupación smoker "Yes" y el segundo es la agrupación smoker "No") y luego los resultados se pegan usando *pandas.concat*, etiquetando las piezas con los nombres de los grupos. 

Por lo tanto, el resultado tiene un índice jerárquico cuyo nivel interno contiene valores de índice del DataFrame original.

Si pasas una función a *apply* que toma otros argumentos o palabras clave, puedes pasarlos después de la función:

In [75]:
prop.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill') # En este caso aplicamos sobre un agrupamiento de dos claves.

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982



---


### Análisis de cuantiles y buckets

Pandas tiene algunas herramientas, en particular *cut* y *qcut*, para dividir los datos en cubos con contenedores de tu elección o por cuantiles de muestra. La combinación de estas funciones con *groupby* hace que sea conveniente realizar análisis de buckets o cuantil en un conjunto de datos. Considere un conjunto de datos aleatorio simple y una categorización de bucket de igual longitud usando cut:

In [ ]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
frame

In [78]:
quartiles = pd.cut(frame.data1, 4) # Cortamos los datos en 4 conjuntos

In [ ]:
quartiles[:10]

El objeto  devuelto por *cut* se puede pasar directamente a *groupby*. Entonces podríamos calcular un conjunto de estadísticas para la columna data2 de la siguiente manera:

In [80]:
def get_stats(group):
  return {'min': group.min(), 'max': group.max(),
          'count': group.count(), 'mean': group.mean()}

In [81]:
grouped = frame['data2'].groupby(quartiles)

In [82]:
grouped.apply(get_stats) # que puedo agregar para que se vea mejor? ..un___..?

data1                  
(-2.793, -1.327]  min       -2.831621
                  max        2.326419
                  count     86.000000
                  mean       0.005667
(-1.327, 0.134]   min       -3.048928
                  max        2.830146
                  count    481.000000
                  mean       0.021828
(0.134, 1.594]    min       -2.703617
                  max        2.935436
                  count    371.000000
                  mean      -0.050832
(1.594, 3.054]    min       -1.678858
                  max        2.476784
                  count     62.000000
                  mean       0.026392
Name: data2, dtype: float64

In [ ]:
# Haz tu magia


### Rellenar valores perdidos con valores específicos de grupo

En clases anteriores vimos que a veces simplemente haremos **dropna** pero otras veces necesitaremos reemplazar los datos faltantes (nulos) por valores convenientes.

*fillna* es la herramienta adecuada para usar; por ejemplo, aquí rellenamos los valores de NA con la media, como vimos previamente:

In [84]:
s = pd.Series(np.random.randn(6))

In [85]:
s[:3] = np.nan
s

0         NaN
1         NaN
2         NaN
3    0.235219
4    0.649135
5   -0.603682
dtype: float64

In [86]:
s.fillna(s.mean())

0    0.093558
1    0.093558
2    0.093558
3    0.235219
4    0.649135
5   -0.603682
dtype: float64

**Supongamos que necesitas que el valor de relleno varíe según el grupo.**

Una forma de hacer esto es agrupar los datos y usar *apply* con una función que llame a *fillna* en cada fragmento de datos. 

Aquí hay algunos datos de muestra sobre los estados de EE. UU. Divididos en regiones orientales y occidentales:

In [87]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

In [96]:
# group_key = ['East'] * 4 + ['West'] * 4 # Notación alternativa
group_key = ['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

Ten en cuenta que la sintaxis ['Este'] * 4 produce una lista que contiene cuatro copias de los elementos en ['Este'].

In [97]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -1.465240
New York      0.873641
Vermont       0.018748
Florida      -1.566175
Oregon       -0.857417
Nevada        0.845183
California   -1.298819
Idaho        -0.525507
dtype: float64

In [98]:
data['Vermont', 'Nevada', 'Idaho'] = np.nan
data

Ohio         -1.465240
New York      0.873641
Vermont            NaN
Florida      -1.566175
Oregon       -0.857417
Nevada             NaN
California   -1.298819
Idaho              NaN
dtype: float64

In [99]:
data.groupby(group_key).mean() # Al hacer el promedio, NO contempla los nulos, no los suma.

East   -0.719258
West   -1.078118
dtype: float64

In [100]:
fill_mean = lambda g: g.fillna(g.mean()) # Que hace esta funcion lambda?

In [101]:
data.groupby(group_key).apply(fill_mean)

Ohio         -1.465240
New York      0.873641
Vermont      -0.719258
Florida      -1.566175
Oregon       -0.857417
Nevada       -1.078118
California   -1.298819
Idaho        -1.078118
dtype: float64


---

## Muestreo aleatorio y permutación
Supongamos que deseas extraer una muestra aleatoria de un gran conjunto de datos para fines de simulación o alguna otra aplicación. Hay varias formas de realizar los "sorteos"; Aquí usamos el método de muestra para Series.

In [102]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names) # Para cada letra itera por la cantidad de cartas

deck = pd.Series(card_val, index=cards)

Así que ahora tenemos una Serie de longitud 52 cuyo índice contiene nombres y valores de cartas que se usan en Blackjack y otros juegos (para simplificar las cosas, solo dejo que el as 'A' sea 1):


In [ ]:
deck[:13]

In [107]:
def draw(deck, n=5): # Esta funcion recibe la serie "Deck" (el mazo) y devuelve una muestra random de "n" elementos
  return deck.sample(n)

In [110]:
draw(deck)

7D    7
6C    6
9S    9
4C    4
6S    6
dtype: int64

Supongamos que quieres dos cartas al azar de cada palo. Debido a que el palo es el último caracter de cada nombre de tarjeta, podemos agruparlo en base a esto y usar apply:

In [123]:
get_suit = lambda card: card[-1] # Tomo la última letra que es el palo

10

In [126]:
deck.groupby(get_suit).apply(draw, n=2)

C  7C      7
   8C      8
D  2D      2
   6D      6
H  10H    10
   5H      5
S  6S      6
   9S      9
dtype: int64



---


### Un ejemplito de transformación y correlación entre columnas

Más adelante vamos a ver más a fondo el tema de correlación, cuando entremos a algoritmos de machine learning, pero veamos ahora una simple transformación.

Consideremos un conjunto de datos financieros originalmente obtenido de Yahoo! Finance que contiene precios al final del día para algunas acciones y el índice S&P 500 (el símbolo SPX):

In [138]:
close_px = pd.read_csv('https://raw.githubusercontent.com/al34n1x/DataScience/master/6.Gestion_de_datos/stocks.csv', 
                       parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [139]:
close_px

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93
...,...,...,...,...
2011-10-10,388.81,26.94,76.28,1194.89
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66


Una tarea de interés podría ser calcular un DataFrame que consta de las correlaciones anuales de los rendimientos diarios con SPX. 

1° paso: Vamos a hacer una **transformación**:


In [140]:
rets = close_px.pct_change().dropna() #Calculamos el procentaje de cambio y eliminamos nulos
# Por defecto, la función pct_change, calcula el porcentaje de cambio entre el valor actual y el de la row inmediata anterior

Vemos que en lugar de los valores originales, ahora tenemos el porcentaje de cambio

¿Qué dato desapareció? ¿Tiene sentido?

In [141]:
rets 

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
...,...,...,...,...
2011-10-10,0.051406,0.026286,0.036977,0.034125
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974


2° paso: Creamos una función que calcula la correlación por pares de cada columna con la columna 'SPX':

In [146]:
spx_corr = lambda x: x.corrwith(x['SPX']) #Esta funcion aplica a Dataframes exclusivamente y mide la correlación entre cada columna con la que se pasa como parámetro

In [169]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year) # Agrupamos los porcentajes de cambio por año
resultado = by_year.apply(spx_corr)
'''
Llama a la funcion spx_corr para calcular la correlación de cada columna del dataframe contra
la columna 'SPX', luego de hacer la agrupacion por año.
'''
resultado

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


Si quisieramos ver la pinta que tiene una matriz de correlación completa para todas las columnas para alguno de los años, por ejemplo para 2003 (primer fila), podemos hacer:

In [183]:
resultado = resultado.loc[2003,:].to_frame().transpose() #Hago el loc que devuelve una serie y la transformo en dataframe
# Agrego "transpose" para obtener las filas como columnas

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0


In [184]:
#Ahora puedo calcular la matriz:
resultado.corr()

,AAPL,MSFT,XOM,SPX
AAPL,1.000000,0.737826,0.903539,0.043359
MSFT,0.737826,1.000000,0.823852,0.642828
XOM,0.903539,0.823852,1.000000,0.147927
SPX,0.043359,0.642828,0.147927,1.000000




---


## Pivot Tables y tabulación cruzada

Una tabla dinámica es una herramienta de resumen de datos que se encuentra con frecuencia en programas de hojas de cálculo. 

Agrega una tabla de datos por una o más claves, organizando los datos en un rectángulo con algunas de las claves de grupo a lo largo de las filas y algunas a lo largo de las columnas. 

Las tablas dinámicas en Python con Pandas son posibles a través de la función *groupby*. DataFrame tiene un método *pivot_table* y también hay una función *pandas.pivot_table* de nivel superior. Además de proporcionar una interfaz conveniente para *groupby*, *pivot_table* puede agregar totales parciales, también conocidos como márgenes.

Volviendo al conjunto de datos de propinas, supongamos que deseamos calcular una tabla de promedios grupales:

In [197]:
prop.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [188]:
prop.pivot_table(index=['day', 'smoker'])
# En este caso estamos generando una agrupación con promedios por columna, 
# y lo que obtenemos es un dataframe con índices jerárquicos

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

Ahora, supongamos que queremos agregar solo *tip_pct* y *size*, y además agrupar por tiempo. 

Pondremos fumador en las columnas de la tabla y día en las filas:

In [198]:
prop.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [199]:
prop.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0) # Si hay NaN podemos usar fill_value

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706


### Tabulaciones cruzadas (crosstab)
Una tabulación cruzada es un caso especial de una tabla dinámica que **calcula las frecuencias de grupo**. Aquí hay un ejemplo:

In [200]:
pd.crosstab([prop.time, prop.day], prop.smoker)

smoker       No  Yes
time   day          
Dinner Fri    3    9
       Sat   45   42
       Sun   57   19
       Thur   1    0
Lunch  Fri    1    6
       Thur  44   17

Podríamos aumentar esta tabla para incluir totales parciales pasando 'margins=true'. Esto tiene el efecto de agregar todas las etiquetas de fila y columna, siendo los valores correspondientes las estadísticas de grupo para todos los datos dentro de un solo nivel

In [201]:
pd.crosstab([prop.time, prop.day], prop.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244